In [ ]:
from dotenv import load_dotenv
from datetime import datetime

from langchain_teddynote import logging
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

from operator import itemgetter

In [ ]:
load_dotenv()

In [ ]:
logging.langsmith("langchain-01")  # 프로젝트 이름

In [ ]:
# prompt 와 llm 을 생성합니다.
prompt = PromptTemplate.from_template("{num} 의 10배는?")
llm = ChatOpenAI(temperature=0)

# chain 을 생성합니다.
chain = prompt | llm

In [ ]:
chain.invoke(5)  # chain.invoke({"num": 5})

In [ ]:
RunnablePassthrough().invoke({"num": 10})

In [ ]:
# RunnablePassthrough로 체인 구성

runnable_chain = {"num": RunnablePassthrough()} | prompt | ChatOpenAI()

In [ ]:
runnable_chain.invoke(10)

In [ ]:
# RunnablePassthrough.assign() : 입력된 값의 키-값 쌍과 새롭게 할당된 키-값 쌍을 결합

(RunnablePassthrough.assign(new_num=lambda x: x["num"] * 3)).invoke({"num": 1})

RunnableParallel

In [ ]:
runnable = RunnableParallel(
    passed = RunnablePassThrough(), 
    extra = RunnablePassthrough.assign(mult=lambda x: x["num"] * 3), 
    modified = lambda x: x["num"] + 1
)

In [ ]:
runnable.invoke({"num": 1})

In [ ]:
chain1 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("{country} 의 수도는?")
    | ChatOpenAI()
)
chain2 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("{country} 의 면적은?")
    | ChatOpenAI()
)

In [ ]:
combined_chain = RunnableParellel(capital=chain1, area=chain2)
combined_chain.invoke("대한민국")

RunnableLambda

In [ ]:
# 사용자 정의 함수 맵핑하기

def get_today(a):  # 오늘 날짜 가져오기
    return datetime.today().strftime("%b-%d")  # '월-일' 형식

get_today(None)  # 오늘 날짜 출력

In [ ]:
prompt = PromptTemplate.from_template(
    "{today} 가 생일인 유명인 {n} 명을 나열하세요. 생년월일을 표기해 주세요."
)

llm = ChatOpenAI(
    temperature=0, 
    model_name="gpt-4.1-mini"
)

In [ ]:
chain = (
    {"today": RunnableLambda(get_today), "n": RunnablePassthrough()} 
    | prompt 
    | llm 
    | StrOutputParser()
)

In [ ]:
print(chain.invoke(3))

itemgetter

In [ ]:
def length_function(text):  # 문장 길이
    return len(text)

def _multiple_length_function(text1, text2):  # 두 문장의 길이를 곱한 값
    return len(text1) * len(text2)

def multiple_length_function(_dict):  # _multiple_length_function 함수를 사용하여 두 문장의 길이를 곱한 값을 반환
    return _multiple_length_function(_dict["text1"], _dict["text2"])

In [ ]:
prompt = ChatPromptTemplate.from_template("{a} + {b} 는 무엇인가요?")
llm = ChatOpenAI()

In [ ]:
chain = (
    {
        "a": itemgetter("word1") | RunnableLambda(length_function),
        "b": {"text1": itemgetter("word1"), "text2": itemgetter("word2")} | RunnableLambda(multiple_length_function),
    }
    | prompt 
    | llm
)

In [ ]:
chain.invoke({"word1": "hello", "word2": "world"})